# CEDR : Contextualized Embeddings for Document Ranking

PACRR-DRMM에 이어서 2019년 이후 현제 가장 성능이 좋은 모델입니다. 모델의 구조는 이전 방식들을 그대로 사용하지만 단어 표현을 word2vec, Glove에서 ELMo, BERT를 적용하였습니다.

## Introduction

NLP Task에서 성능을 높히는 가장 일반적인 방법중 하나는 단어의 표현 벡터를 더 잘 구하는 것입니다. 이전의 PACRR, DRMM에서는 Word2Vec, Glove를 사용하였습니다. 논문은 ELMo, BERT와 같이 Contextual을 고려하는 방법을 사용하면 성능 향상을 이루어 낼수 있다고 가설을 세운다음 여러가지 방법을 적용하여 성능향상을 실험적으로 규명하였습니다.

## Methodology



### Notation

기존의 논문들과 같은 표기법을 사용합니다.

### Contextualized similarity tensors

논문에서 강조하는 contextualized representation를 제시하는 파트 입니다. 일반적으로Contextualized language model은 multiple stacked layers를 포함합니다. 직관적으로 layer가 깊을수록 더 많은 컨텍스트가 통합되므로 이를 랭킹모델에 학습시키기 위하여 각 레이어당 유사도를 계산하여 3차원 유사도 표현을 만듭니다. 이에 논문은 contextualized representation을 다음과 같이 정의 합니다.

$$ S_{Q,D}[l, q, d] = cos(context_{Q, D}(q, l), context_{Q, D}(d, l))$$

여기서 $q \in Q, d \in D$는 각 검색어와 검색결과의 단어들입니다. PACRR과 같은 n-gram based model은 multiple input으로, DRMM과 같은 counting based model은 각 채널에서 갯수를 새어 통합합니다.

###  Joint BERT approach

BERT의 미세조정 과정에서 [CLS] 토큰은 두번째문장을 예측하는 정보를 가지고 있으므로 중요한 feature가 될 수 있다고 보고 이 정보를 활용하기 위하여 검색어 term에 [CLS] 토큰을 포함하여 유사도를 계산해야 한다고 제안했습니다. 이후 실험 결과에서 [CLS]토큰의 포함 여부가 상당한 성능 차이를 보이며 [CLS]토큰의 중요성을 입증하였습니다.

## Experiment

논문에서의 결과는 ELMo와 BERT의 임베딩표현만을 사용한경우에는 성능 향상이 조금 이루어 졌지만 미세조정을 거친 BERT와 [CLS]토큰을 포함한 방법은 상당한 성능 향상을 가져왔습니다. 제가 직접 joint BERT 방법을 실험해보고 이전 결과와 비교해 보겟습니다.

먼저 BERT모델링을 한후 pretrain과 fine tunning을 직접해보려 하였으나 구현후 pretrain 하는순간 멍청한 생각이란걸 알았습니다... 어떻게 GPU도 없는 노트북에서 BERT를 훈련시킬 생각을 한걸까요? 따라서 구글에서 제공하는 pretrained bert의 weight를 받아서 미세조정을 하기로 하였습니다.

미세 조정은 이전에 했던 검색결과 만족도 판별로 하였습니다. 여기서 한가지 문제가 또 발생하는데, 역시나 imbalance data... 또 저를 괴롭힙니다. 여러 논문들을 찾아본결과 kappa, focal loss를 사용하는 방법, data Augmentation, weighted cross entropy정도등의 해결책이 있었습니다. 그중 가장 효과가 좋다고 하는 weighted cross entropy로 훈련시킨 결과 kappa score 0.6점 이상으로 만족할만한 성능이 나왔습니다.(사실 모두 시도해 보았지만 weighted cross entropy가 가장 성능이 좋았습니다.) 추가로 이전 모델과 soft voting을 통해 0.3의 성능 향상이 있었습니다. 훈련과정은 BERT.ipynb를 참고해 주세요

미세 조정을 마친 후 [CLS]토큰을 포함한 유사도 행렬로 DRMM, PACRR, DRMM-PACRR에 적용시켜보았습니다.

<img src='./img/drmm_bert_loss.png'>

<img src='./img/drmm_result.png'>

먼저 DRMM입니다. 훈련은 안정되게 진행되었습니다. 그러나 이전에 말했듯이 수치로 보는것보다 예시가 직관적입니다. 훈련과정 그래프 보다 이전에 예시로 들었던 검색어 nike flip flops를 기준으로 비교해 보겠습니다. 먼저 $nDCG@20 = $, $mAP@20 = $으로 확실히 이전의 모델보다 높은 결과입니다. 하지만 직관적으로 관련성이 4점인 제품들이 아래에도 분포하고 있어 썩 만족스럽지는 않습니다.

<img src='./img/pacrr_bert_loss.png'>

<img src='./img/pacrr_drmm_result.PNG'>

다음은 PACRR여전히 DRMM과 비슷한 성능입니다. 평가 점수로는 $nDCG@20 = 0.93$, $mAP@20 = 0.84$입니다. DRMM과 다르게 하지만 연산량이 많아짐에 따라 학습속도가 느립니다. 

<img src='./img/drmm_pacrr_bert_loss.png'>

<img src='./img/pacrr_result.png'>

마지막 PACRR-DRMM입니다. 훈련과정이 안정적인 모습을 보여주면서 평가 점수 $nDCG@20 = 0.91$, $mAP@20 = 0.81$로 가장 성능이 좋습니다.

## Discussion

검색어-검색결과 예측 모델을 만들다 보니 자연스럽게 추천 모델과 검색 구조에 대하여 공부하게 되었습니다. 현제 추천시스템의 추세는 크게 유저의 행동 데이터, 제품의 카테고리 정보, 랭킹 정보 3가지를 하나로 모아 모델링을 합니다. 구현한 논문들은 아마존, 이베이의 검색 시스템 관련한 랭킹 시스템 논문의 인용자료중 하나로 DNN을 기반으로한 랭킹모델에 대한 정의와 요구 조건을 자세히 정리되어 있습니다. 그중 제품의 관련성(Semantic Matching part)에 대하여 잘 설명되어 있는 DRMM 부터 같은 데이터로 최근 SOTA인 CEDR까지 주어진 검색 결과 데이터를 기반으로 랭킹 모델을 구현하였습니다. crowdflower data로 모델을 적용해본 결과 아키텍쳐간 성능 차이는 적지만 bert를 활용한 방법은 성능 향상이 눈에 보이며 최근 트렌드와 같이 제품의 상세 정보 카테고리(브랜드, 색상, 용도 등등)와 보